In [18]:
# Basit bir fonksionumuz olduğunu düşünelim

def topla(X:int,Y:float) -> float:
    """X ye Y rakamlarını toplar."""
    return X+Y

In [ ]:
# Fonksiyonu test edelim
topla(X=3,Y=4.3) # Outputs: 7.3

In [ ]:
# Hatalı kullanalım
topla(X=1.2,Y=3) # Outputs: 4.2

#### TIP KONTROLU

Tipleri kontrol etmediğimiz için, int yerine float versek dahi fonksiyonumuz çalışıyor.

Ne yapabiliriz?

##### Method 1:
`ensure` kütüphanesi kullanılabilir.


In [ ]:
# `ensure` kitiphanesini dahil edelim
from ensure import ensure_annotations

# Fonksiyonumuzu tekrar tanımlayalım. Dekoratörü'ümüzü unutmayalım.
@ensure_annotations
def topla(X:int,Y:float) -> float:
    """X ye Y rakamlarını toplar."""
    return X+Y


In [ ]:
# Doğru kullanım
print(f"Doğru: {topla(X=3,Y=4.3)}") # Outputs: 7.3
# Hatalı kullanalım
print(f"Yanlış: {topla(X=1.2,Y=3)}") # Outputs: Error

Yanlış kullanımda hata aldık ama fonksiyon içerisinde belirlediğimiz docstringimiz kayboldu ve 

**"(function) topla: WrappedFunctionReturn | Wrapped"**

 şeklinde görünüyor.

##### Method 2:
`isinstance` sabiti kullanılabilir.

In [ ]:

def topla(X:int,Y:float) -> float:
    """X ye Y rakamlarını toplar."""
    assert isinstance(X,int), "X argümanı int değil."
    assert isinstance(Y,float), "Y argümanı float değil."
    result = X+Y
    assert isinstance(result,float),"Function float döndürmüyor"
    return result

# Doğru kullanım
print(f"Doğru: {topla(X=3,Y=4.3)}") # Outputs: 7.3
# Hatalı kullanalım
print(f"Yanlış: {topla(X=1.2,Y=3)}") # Outputs: Error

##### Method 3:
`type` ve `is` ile karşılaştırma yapılarak assert edilebilir.

In [ ]:
def topla(X:int,Y:float) -> float:
    """X ye Y rakamlarını toplar."""
    assert type(X) is int, "X argümanı int değil."
    assert type(Y) is float , "Y argümanı float değil."
    result = X+Y
    assert type(result) is float, "Function float döndürmüyor"
    return result

# Doğru kullanım
print(f"Doğru: {topla(X=3,Y=4.3)}") # Outputs: 7.3
# Hatalı kullanalım
print(f"Yanlış: {topla(X=1.2,Y=3)}") # Outputs: Error

##### Method 4:

`issubclass` kullanılabilir.

In [ ]:
def topla(X:int,Y:float) -> float:
    """X ye Y rakamlarını toplar."""
    assert issubclass(type(X),int), "X argümanı int değil."
    assert issubclass(type(Y),float) , "Y argümanı float değil."
    result = X+Y
    assert issubclass(type(result),float), "Function float döndürmüyor"
    return result

# Doğru kullanım
print(f"Doğru: {topla(X=3,Y=4.3)}") # Outputs: 7.3
# Hatalı kullanalım
print(f"Yanlış: {topla(X=1.2,Y=3)}") # Outputs: Error

##### Method 5:

`type` ile tip isminden (`__name__ `) kıyaslanabilir.

In [ ]:
def topla(X:int,Y:float) -> float:
    """X ye Y rakamlarını toplar."""
    assert type(X).__name__ == "int", "X argümanı int değil."
    assert type(Y).__name__ == "float" , "Y argümanı float değil."
    result = X+Y
    assert type(result).__name__ == "float" , "Function float döndürmüyor"
    return result

# Doğru kullanım
print(f"Doğru: {topla(X=3,Y=4.3)}") # Outputs: 7.3
# Hatalı kullanalım
print(f"Yanlış: {topla(X=1.2,Y=3)}") # Outputs: Error

##### Method 6:

Custom Function: type_checker as decorator

In [9]:
from typing import get_type_hints

def type_checker(f):
    """
    A decorator that verifies if the arguments of a function match the specified type.
    @param f: the function
    @return:
    """
    def checker(*args, **kwargs):
        hints = get_type_hints(f)

        all_args = kwargs.copy()
        all_args.update(dict(zip(f.__code__.co_varnames, args)))

        # Check the args type
        for arg_name, arg_value in all_args.items():
            if arg_name in hints:
                expected_type = hints[arg_name]
                # If Union get values
                if hasattr(expected_type, "__origin__") and expected_type.__origin__ == typing.Union:
                    expected_type = expected_type.__args__
                msg = f"The `{arg_name}` parameter in `{f.__name__}` function should be an {expected_type} type. " \
                    f"A {type(arg_value)} type was passed instead."
                assert isinstance(arg_value, expected_type), msg

        output = f(*args, **kwargs)

        # Check the output type if specified
        output_type = hints.get('return', None)
        if output_type:
            msg = f"The `{f.__name__}` function should return a {output_type}, " \
                f"but it's returning a {type(output)} instead. " \
                "Please ensure that the return type matches the expected type."
            assert isinstance(output, output_type), msg
        return output
    return checker

In [ ]:
@type_checker
def topla(X:int,Y:float) -> float:
    """X ye Y rakamlarını toplar."""
    return X+Y

# Doğru kullanım
print(f"Doğru: {topla(X=3,Y=4.3)}") # Outputs: 7.3
# Hatalı kullanalım
print(f"Yanlış: {topla(X=1.2,Y=3)}") # Outputs: Error

In [ ]:
from typing import get_type_hints

def topla(X:int,Y:float) -> float:
    """X ye Y rakamlarını toplar."""
    hints = get_type_hints(topla)
    print(hints)
    print(X)
    return X+Y

topla(3,5)